In [1]:
import glob
import uqer
import time
import joblib
import pickle
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from uqer import DataAPI   #优矿api
from sklearn import linear_model
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from pandas.core.common import SettingWithCopyWarning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [2]:
result = pd.read_csv('pct1_cal/full/full_Alter_extra_224_LGBMRegressor_openclose_pct1_full_rank_r2022-07-25:11:08:34_train_0.157.csv')

In [6]:
result

,ticker,tradeDate,y,y_pred
0,1,2019-01-02,0.863836,0.539390
1,1,2019-01-03,0.891781,0.495638
2,1,2019-01-04,0.065479,0.430368
3,1,2019-01-07,0.516977,0.495471
4,1,2019-01-08,0.856517,0.450997
...,...,...,...,...
3054427,689009,2022-03-28,0.912723,0.530849
3054428,689009,2022-03-29,0.058873,0.472546
3054429,689009,2022-03-30,0.425688,0.513223
3054430,689009,2022-03-31,0.188525,0.541769


In [3]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054432 entries, 0 to 3054431
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ticker     int64  
 1   tradeDate  object 
 2   y          float64
 3   y_pred     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 93.2+ MB


In [5]:
result.index

RangeIndex(start=0, stop=3054432, step=1)

In [8]:
result.values

array([[1, '2019-01-02', 0.8638356164383562, 0.5393900800523802],
       [1, '2019-01-03', 0.8917808219178082, 0.495637580550581],
       [1, '2019-01-04', 0.0654794520547945, 0.4303680153611025],
       ...,
       [689009, '2022-03-30', 0.4256884591128863, 0.5132234546025178],
       [689009, '2022-03-31', 0.1885245901639344, 0.541769417108376],
       [689009, '2022-04-01', 0.2077117041395251, 0.5557608565406398]],
      dtype=object)

In [9]:
result.describe()

,ticker,y,y_pred
count,3.054432e+06,3.054432e+06,3.054432e+06
mean,3.234161e+05,4.992790e-01,4.987536e-01
std,2.685459e+05,2.865787e-01,4.525975e-02
min,1.000000e+00,2.101282e-04,9.806987e-02
25%,2.535000e+03,2.515602e-01,4.794022e-01
50%,3.005570e+05,4.988745e-01,5.047842e-01
75%,6.008260e+05,7.469619e-01,5.232146e-01
max,6.890090e+05,1.000000e+00,7.137872e-01


In [18]:
result.loc[0:4, 'ticker':'y']

,ticker,tradeDate,y
0,1,2019-01-02,0.863836
1,1,2019-01-03,0.891781
2,1,2019-01-04,0.065479
3,1,2019-01-07,0.516977
4,1,2019-01-08,0.856517
